In [4]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

In [6]:
def utc_to_unix(date):
    '''Forces a timestamp into the UTC timezone and converts it to a UNIX epoch'''
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())

def unix_to_utc(unix):
    '''Converts a UNIX epoch to a UTC Timestamp'''
    return datetime.datetime.utcfromtimestamp(unix).strftime('%Y-%m-%d %H:%M:%S')
#prepare the API call
url = "https://api.pushshift.io/reddit/submission/search/"
subreddit = 'nosleep' #/r/nosleep
fields = ['author', 'subreddit','created_utc','num_comments','score','title','selftext']
sort_type = 'created_utc'
sort = 'asc'
size = 500 

#Declare start and end of reddit posts to extract 
start_date = dt.datetime.strptime("2020-04-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-05-01", "%Y-%m-%d")

URL = "https://api.pushshift.io/reddit/submission/search/"  #query submissions
PARAMS = {
    'after': utc_to_unix(start_date)-1, #get dates after April 1, 2020 UTC
    'before': utc_to_unix(end_date), #get dates before May 1, 2020 UTC
    'sort_type': sort_type, # sort by created_utc
    'sort': sort, # sort in descending order
    'subreddit': subreddit, # do a search on ProRevenge subreddit
    'size': size, # give only 500 search results
    'fields': fields  #return only the following fields
}

In [7]:
#Setup blank list
results = []

#loop while date range not fulfilled
while PARAMS['after'] < PARAMS['before']:
    #use the requests library to query pushshift api
    r = requests.get(url = URL, params = PARAMS)
    
    if r.json()['data'] == []:
        break
        
    #extend list results
    results.extend(r.json()['data'])

    #change start_time
    start_date = r.json()['data'][-1]['created_utc'] # this sets new start time to the last timestamp in the result array
    PARAMS['after'] = start_date
    print('Done until {} - Result Size {} - Total Results Size {}'.format(unix_to_utc(start_date), len(r.json()['data']), len(results)))
    time.sleep(1)

Done until 2020-04-01 16:36:19 - Result Size 100 - Total Results Size 100
Done until 2020-04-02 08:35:51 - Result Size 100 - Total Results Size 200
Done until 2020-04-03 00:42:32 - Result Size 100 - Total Results Size 300
Done until 2020-04-03 18:37:58 - Result Size 100 - Total Results Size 400
Done until 2020-04-04 15:12:40 - Result Size 100 - Total Results Size 500
Done until 2020-04-05 06:34:08 - Result Size 100 - Total Results Size 600
Done until 2020-04-06 00:43:33 - Result Size 100 - Total Results Size 700
Done until 2020-04-06 18:50:07 - Result Size 100 - Total Results Size 800
Done until 2020-04-07 14:46:39 - Result Size 100 - Total Results Size 900
Done until 2020-04-08 07:37:51 - Result Size 100 - Total Results Size 1000
Done until 2020-04-09 04:12:01 - Result Size 100 - Total Results Size 1100
Done until 2020-04-09 21:34:07 - Result Size 100 - Total Results Size 1200
Done until 2020-04-10 13:46:35 - Result Size 100 - Total Results Size 1300
Done until 2020-04-11 04:42:49 - R

In [10]:
df = pd.json_normalize(results)
#saves the dataframe for sanity
df.to_csv('reddit_nosleep.csv', index=False)

In [13]:
df = pd.read_csv('reddit_nosleep.csv')
df.head()

,author,created_utc,num_comments,score,selftext,subreddit,title
0,SpOoKyCaT--,1585699665,8,5,Let me preface this by saying I just moved int...,nosleep,My Mom Checks In On Me When I Sleep
1,ak4402,1585701329,2,2,[removed],nosleep,It was just before sunrise...
2,[deleted],1585701960,2,1,NaN,nosleep,The Boy Disappeared into the forest...
3,quebrain,1585702604,4,1,Mother told me to wash the dishes. I complied....,nosleep,We Can Be Better
4,[deleted],1585703084,0,1,NaN,nosleep,Every single passenger in this train is going ...


In [14]:
# How many submissions were you able to gather?
total_submissions = len(df)
print('Total April 2020 Submissions: {}'.format(total_submissions))

Total April 2020 Submissions: 4265


In [18]:
# Who has the most submissions?
most_active = df['author'].value_counts().reset_index()[0:2]
print('Author with most submissions for April 2020 in the subreddit: {}'.format(most_active['index'][0]))

Author with most submissions for April 2020 in the subreddit: [deleted]


In [19]:
# Which submission has the highest score?
top_submission = df.sort_values(by=['score'],ascending=False,ignore_index=True).head(1)
print('The top submission of April 2020 in the subreddit is: \n\n{} by {}'.format(top_submission['title'][0],top_submission['author'][0]))

The top submission of April 2020 in the subreddit is: 

Working at an amusement park: unsuccessful backstabbing by girl_from_the_crypt


In [21]:
#Which submission has the highest number of comments?
top_commented = df.sort_values(by=['num_comments'],ascending=False,ignore_index=True).head(1)
print('The post with the highest number of comments for April 2020 in the subreddit is: \n\n{} by {}'.format(top_commented['title'][0],top_commented['author'][0]))

The post with the highest number of comments for April 2020 in the subreddit is: 

Working at an amusement park: kiss, swallow, turn by girl_from_the_crypt


In [22]:
# Which day of the week has the most submissions?
df['dayname'] = pd.to_datetime(df['created_utc'], unit='s').dt.day_name() #creates dayname column
most_active_day = df['dayname'].value_counts().reset_index()[0:2]
print('Most active day in April 2020: {}'.format(most_active_day['index'][0]))

Most active day in April 2020: Thursday
